In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,605 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
from pyspark.sql import  SparkSession

spark = SparkSession.builder \
                            .master("local[3]") \
                            .appName("firstSparkRDDExemple") \
                            .getOrCreate()

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
rdd = spark.sparkContext.parallelize(data)

In [ ]:
rddMap = rdd.map(lambda x:  x+1)
rddMap.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [ ]:
rddPair = rdd.map(lambda x : (x, x*x))
rddPair.collect()

[(1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81),
 (10, 100),
 (11, 121),
 (12, 144)]

In [ ]:
rddFlatMap = rdd.flatMap(lambda x : (x, x*x))
rddFlatMap.collect()

[1,
 1,
 2,
 4,
 3,
 9,
 4,
 16,
 5,
 25,
 6,
 36,
 7,
 49,
 8,
 64,
 9,
 81,
 10,
 100,
 11,
 121,
 12,
 144]

In [ ]:
rddFilter = rdd.filter(lambda x : x>5)
rddFilter.collect()

[6, 7, 8, 9, 10, 11, 12]

In [ ]:
def f(partitionData):
  sum2 = 0
  sum1 = 0
  for element in partitionData:
    if element % 2 == 0:
      sum2 += element
    else:
      sum1 += element
  return [sum2, sum1]

rddr = rdd.repartition(3)
rddMP = rddr.mapPartitions(f)
rddMP.collect()

[30, 25, 12, 11, 0, 0]

In [ ]:
co = rdd.count()
co

12

In [ ]:
sum = rddr.reduce(lambda x, y: x+y)
sum

78

In [ ]:
text = ["bonjour SDIA, bienvenu au module bdpa", \
        "ce module ( bdpa ) sera consacré au technologies big data", \
        "c'est important pour les SDIA par ce que les technologies big data sont devenu primordiales", \
        "j'espere que le module bdpa vous donnéera les connaissances necessaires pour travailler avec les big data"]

lines = spark.sparkContext.parallelize(text)
words = lines.flatMap(lambda line : line.split(" "))
pairWords = words.map(lambda word : (word, 1))
count = pairWords.reduceByKey(lambda x, y : x+y)
count.collect()

[('bonjour', 1),
 ('SDIA,', 1),
 ('bienvenu', 1),
 ('au', 2),
 ('module', 3),
 ('bdpa', 3),
 ('ce', 2),
 ('(', 1),
 (')', 1),
 ('sera', 1),
 ('consacré', 1),
 ('technologies', 2),
 ('big', 3),
 ('data', 3),
 ("c'est", 1),
 ('important', 1),
 ('pour', 2),
 ('les', 4),
 ('SDIA', 1),
 ('par', 1),
 ('que', 2),
 ('sont', 1),
 ('devenu', 1),
 ('primordiales', 1),
 ("j'espere", 1),
 ('le', 1),
 ('vous', 1),
 ('donnéera', 1),
 ('connaissances', 1),
 ('necessaires', 1),
 ('travailler', 1),
 ('avec', 1)]

In [ ]:
co = pairWords.countByKey()
co

defaultdict(int,
            {'bonjour': 1,
             'SDIA,': 1,
             'bienvenu': 1,
             'au': 2,
             'module': 3,
             'bdpa': 3,
             'ce': 2,
             '(': 1,
             ')': 1,
             'sera': 1,
             'consacré': 1,
             'technologies': 2,
             'big': 3,
             'data': 3,
             "c'est": 1,
             'important': 1,
             'pour': 2,
             'les': 4,
             'SDIA': 1,
             'par': 1,
             'que': 2,
             'sont': 1,
             'devenu': 1,
             'primordiales': 1,
             "j'espere": 1,
             'le': 1,
             'vous': 1,
             'donnéera': 1,
             'connaissances': 1,
             'necessaires': 1,
             'travailler': 1,
             'avec': 1})

In [ ]:
clusteringData = [[0.0, 0.0, 0.0],
[0.1, 0.1, 0.1],
[0.2, 0.2, 0.2],
[9.0, 9.0, 9.0],
[9.1, 9.1, 9.1],
[9.2, 9.2, 9.2]]

In [ ]:
import numpy as np

# Function to calculate the Euclidean distance
def euclidean_distance(point, centroid):
    point = np.array(point) # Convert point to a NumPy array
    centroid = np.array(centroid)
    return np.sqrt(np.sum((point - centroid) ** 2))

# Function to assign a point to the nearest centroid
def assign_to_centroid(point, centroids):
    distances = [euclidean_distance(point, centroid) for centroid in centroids]
    return np.argmin(distances)

# Function to update centroids
def compute_centroids(points):
    num_points = points.count()
    sum_points = points.reduce(lambda p1, p2: p1 + p2)
    return sum_points / num_points if num_points > 0 else None

# Initialize parameters
k = 2   # Number of clusters
max_iter = 5   # Maximum number of iterations
tol = 0.001     # Convergence tolerance

In [ ]:
data = spark.sparkContext.parallelize(clusteringData)
centroids = data.takeSample(False, k)

for i in range(max_iter):
  clusters = data.map(lambda point: (assign_to_centroid(point, centroids), (point, 1)))

  new_centroids = clusters \
        .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
        .mapValues(lambda v: [x / v[1] for x in v[0]])\
        .collect()

  # Update centroids and check for convergence
  new_centroids = [centroid for idx, centroid in sorted(new_centroids)]
  centroids = new_centroids

print("Final centroids:", centroids)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 19) (e4344a8efb67 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/di